# Polyglot Persistence
# NoSQL(Redis) with python

## Food basket and Item status stored in Redis database

In [ ]:
# Import Redis package
import redis,uuid

# Main class Redis() which you use to execute Redis commands (the port and db=0 are default values)
# Localhost = 127.0.0.1
r8 = redis.Redis(host='localhost', port=6379, db=8)

# Check database connection -will return true if successful
print(r8.ping())

In [ ]:
import random


# Redis hash of field-value pairs is used. Eash hash has a key that with an integer (we are importing random())
random.seed(203)

import random
# Redis hash of field-value pairs is used. Eash hash has a key that with an integer (we are importing random())
random.seed(203)

# The prefix food_basket creates a namespace. We are keeing only food_basket created in Redis
food_baskets = {f"food_basket:{random.getrandbits(32)}": i for i in (
    {
        'customer_id':1,
        'order_id':1,
        'total_amount':4000,
        'quantity:1':4,
        'quantity:2':2,
        'menu_name:1':'chicken momos',
        'menu_status:1':1,
        'menu_status:2':0,
        'menu_name:2':'chicken satay',
        'site_name' : 'Party Fowl',
        'date':'22 may, 2020',
        'time':'14:00',
        'discount_coupon': 'TURKEY25',
        'discount_percentage':25,
        'amount_after_discount':3000 
    },
    {
        'customer_id':2,
        'order_id':2,
        'total_amount':5000,
        'quantity:1':4,
        'quantity:2':2,
        'menu_name:1':'banana',
        'menu_name:2':'apples',
        'menu:1_status':1,
        'menu:2_status':0,
        'site_name' : 'Like No Udder',
        'date':'21 may, 2020',
        'time':'14:00',
        'discount_coupon': 'ADIDAS30',
        'discount_percentage':30,
        'amount_after_discount':3500 
    },
    {
        'customer_id':3,
        'order_id':3,
        'total_amount':7000,
        'quantity:1':4,
        'quantity:2':2,
        'menu_name:1':'banana',
        'menu_name:2':'apples',
        'menu:1_status':1,
        'menu:2_status':0,
        'site_name' : 'Basic need Pizza',
        'date':'21 may, 2020',
        'time':'14:00',
        'discount_coupon': 'BODYSHOP50',
        'discount_percentage':50,
        'amount_after_discount':3500
    },
    {
        'customer_id':4,
        'order_id':4,
        'total_amount':8000,
        'quantity:1':4,
        'quantity:2':2,
        'menu_name:1':'banana',
        'menu_name:2':'apples',
        'menu:1_status':1,
        'menu:2_status':1,
        'site_name' : 'The Codfather',
        'date':'21 may, 2020',
        'time':'14:00',
        'discount_coupon': 'TURKEY25',
        'discount_percentage':25,
        'amount_after_discount':6000
    },
    {
        'customer_id':2,
        'order_id':2,
        'total_amount':4000,
        'quantity:1':4,
        'quantity:2':2,
        'menu_name:1':'banana',
        'menu_name:2':'apples',
        'menu:1_status':1,
        'menu:2_status':1,
        'site_name' : 'Life of Pie',
        'date':'21 may, 2020',
        'time':'14:00',
        'discount_coupon': 'TURKEY25',
        'discount_percentage':25,
        'amount_after_discount':3000
    })
}





In [ ]:
with r8.pipeline() as pipe:
        for basket_id, food_basket in food_baskets.items():
            pipe.hmset(basket_id, food_basket)   
        pipe.execute()
r8.keys()#displaying all the Redis hash present in the namespace

## TTL(Time to Live) expiration if not proceeded to the next step

#Hash keys will not be added from Redis cache to the database if the food basket stays idle for 10 mins without completing the payment.


In [ ]:
from datetime import datetime
# retrieving current time using using datetime object
now = datetime.now()

In [ ]:
#Time To Live (TTL) implementation on customer's food basket. 
#Use this to temporarily store useful data.
# Every key has TTL associated with it and the default value is -1.
# Set this number to a positive value and which represents the number of seconds remaining before the data expires.

r8.set('food_basket:3616826228',now.strftime("%H:%M:%S"))
r8.ttl('food_basket:3616826228')
r8.expire('food_basket:3616826228', 600) #expiry set for 10 minutes

In [ ]:
r8.ttl('food_basket:3616826228')

In [ ]:
r8.ttl('food_basket:3616826228')

## If the Redis Hash key is not expired, adding it to the database from cache

In [ ]:
# Adding data to database using HMSET

# If the Redis hash key has not expired ADD it to the database using pipelining
# The code block above also introduces the concept of Redis pipelining, which is a way to cut down the number of round-trip transactions that you need to write or read data from your Redis server. 
# If you would have just called r.hmset() three times, then this would necessitate a back-and-forth round trip operation for each row written.

#Using if statement to check if the hash has expired. 
#If hash is not expired, pipelining the data(hash) into the database using HMSET
if r8.ttl('food_basket:3616826228') !=-2:
    with r8.pipeline() as pipe:
        for basket_id, food_basket in food_baskets.items():
            pipe.hmset(basket_id, food_basket)   
        pipe.execute()


In [ ]:
# Saving data

r8.bgsave()

In [ ]:
#displaying keys having food_basket in the name of hashes
r8.keys("food_basket*")

In [ ]:
# View values
#please change the index of hash to one of the values in the output above
print(r8.hget("food_basket:2113567782", "total_amount"))
print(r8.hget("food_basket:2113567782", "discount_percentage"))

## II. Second feature implementation
## Pattern matching 

## Saving another set of hash keys onto a distributed cache to know the Item ingredients( done this to avoid round-trip time)

## Using pattern matching to help users view only those items which they are not allergic to

## Using another namespace menu_items

In [ ]:
import random


# Redis hash of field-value pairs is used. Eash hash has a key that with an integer (we are importing random())
random.seed(203)

import random
# Redis hash of field-value pairs is used. Eash hash has a key that with an integer (we are importing random())
random.seed(203)

menu_items = {f"menu_item:{random.getrandbits(32)}": i for i in (
    {
        'menu_id':1,
        'menu_name':'chicken momos',
        'chicken':'ingredient',
        'carrot':'ingredient',
        'refined flour':'ingredient',
        'cabbage':'ingredient',
        'Oil':'ingredient'
        
    },
    {
        'menu_id':2,
        'menu_name':'veg momos',
        'paneer':'ingredient',
        'mushroom':'ingredient',
        'refined flour':'ingredient',
        'maida':'ingredient'
        
    },
    {
        'menu_id':3,
        'menu_name':'chicken noodles',
        'chicken':'ingredient',
        'soy sauce':'ingredient',
        'Chilli pepper':'ingredient'
    },
    {
        'menu_id':4,
        'menu_name':'pork soup',
        'sodium soy sauce':'ingredient',
        'boneless pork':'ingredient',
        'Sliced mushrooms':'ingredient'
    },
    {
        'menu_id':5,
        'menu_name':'chicken biriyani',
        'ginger garlic paste':'ingredient',
        'chicken':'ingredient',
        'turmeric':'ingredient',
        'chilly powder':'ingredient',
        'garam masala':'ingredient'
    },
    {
        'menu_id':6,
        'menu_name':'Pancakes',
        'eggs':'ingredient',
        'milk':'ingredient',
        'maple syrup':'ingredient',
        'berries':'ingredient'
    })
}




In [ ]:
# Adding data to database using HMSET

# The code block below also introduces the concept of Redis pipelining, which is a way to cut down the number of round-trip transactions that you need to write or read data from your Redis server. 
# If you would have just called r.hmset() three times, then this would necessitate a back-and-forth round trip operation for each row written.


with r8.pipeline() as pipe:
        for menus_id, menu_item in menu_items.items():
            pipe.hmset(menus_id, menu_item)   
        pipe.execute()
        


In [ ]:
#Displays all the keys matching menu_item
r8.keys("menu_item*")

### Filtering keys based on customer's allergy ( Using scan_iter() function)

In [ ]:
"""If customer has allergy of chicken and wants to filter out all the menu_item having chicken
   Using scan_iter() function to find keys in a subset of your keyspace, consider using SCAN
   Scans for ingredients having chicken. If chicken exists in the namespace deletes those keys 
"""
for k in r8.scan_iter("chicken*"):
    r8.delete(k)
#displays us menu_items which don't have chicken from Redis cache. In this case an empty array
#The SCAN comman will show an error on redis-cli version lower than 2.8. Virtual desktop has version 2.4.5

In [ ]:
#Displays all the keys after items which user is allergic to is deleted above
r8.keys("menu_item*")

# If Menu items are Out of Stock!

## Exception handling

In [ ]:
import logging

logging.basicConfig()

class OutOfStockError(Exception):
    """Raised when menu items are all out of stock"""

def buyitem(r8: redis.Redis, itemid: int) -> None:
    with r8.pipeline() as pipe:
        error_count = 0
        while True:
            try:
                # Get available inventory, watching for changes
                # related to this itemid before the transaction
                pipe.watch(itemid)
                nleft: bytes = r.hget(itemid, "menu_status")
                if nleft > b"0":
                    pipe.multi()
                    print('Item back in stock')
                    pipe.execute()
                    break
                else:
                    # Stop watching the itemid and raise to break out
                    pipe.unwatch()
                    raise OutOfStockError(
                        f"Sorry, {itemid} is out of stock!"
                    )
            except redis.WatchError:
                # Log total num. of errors by this user to buy this item,
                # then try the same process again of WATCH/HGET/MULTI/EXEC
                error_count += 1
                logging.warning(
                    "WatchError #%d: %s; retrying",
                    error_count, itemid
                )
    return None

#This doesn't return any error in our keyspace as all the items are in stock